1. Configurar o Darknet (YOLO):

In [1]:
# Clonar o repositório darknet
!git clone https://github.com/AlexeyAB/darknet

# Ir para o diretório darknet
%cd darknet

# Compilar o darknet
!make


Cloning into 'darknet'...
remote: Enumerating objects: 15873, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 15873 (delta 12), reused 7 (delta 7), pack-reused 15850 (from 3)
Receiving objects: 100% (15873/15873), 14.50 MiB | 14.97 MiB/s, done.
Resolving deltas: 100% (10679/10679), done.
/content/darknet
mkdir -p ./obj/
mkdir -p backup
mkdir -p results
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -c ./src/image_opencv.cpp -o obj/image_opencv.o
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -c ./src/http_stream.cpp -o obj/http_stream.o
./src/http_stream.cpp: In member function ‘bool JSON_sender::write(const char*)’:
./src/http_stream.cpp:253:21: warning: unused variable ‘n’ []8;;https://gcc.gnu.org/onlinedocs/gcc/

Passo 2: Baixar os Pesos Pré-treinados

In [2]:
# Baixar pesos pré-treinados
!wget https://pjreddie.com/media/files/yolov3.weights

--2024-12-19 02:06:56--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 162.0.215.52
Connecting to pjreddie.com (pjreddie.com)|162.0.215.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  15.0MB/s    in 16s     

2024-12-19 02:07:13 (14.8 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]



Passo 3: Preparar o Dataset

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

MessageError: Error: credential propagation was unsuccessful

In [4]:
import json
import os

def convert_labelme_to_yolo(json_file, output_path):
    with open(json_file, 'r') as f:
        data = json.load(f)

    img_w = data['imageWidth']
    img_h = data['imageHeight']

    with open(output_path, 'w') as f:
        for shape in data['shapes']:
            points = shape['points']
            label = shape['label']

            x_min = min([p[0] for p in points]) / img_w
            y_min = min([p[1] for p in points]) / img_h
            x_max = max([p[0] for p in points]) / img_w
            y_max = max([p[1] for p in points]) / img_h

            x_center = (x_min + x_max) / 2
            y_center = (y_min + y_max) / 2
            width = x_max - x_min
            height = y_max - y_min

            f.write(f"{label} {x_center} {y_center} {width} {height}\n")

# Caminho para o diretório de anotações e de saída
annotations_dir = '/content/gdrive/My Drive/dataset/labels'
output_dir = '/content/gdrive/My Drive/dataset/yolo_labels'
os.makedirs(output_dir, exist_ok=True)

# Converter todas as anotações
for annotation_file in os.listdir(annotations_dir):
    json_path = os.path.join(annotations_dir, annotation_file)
    output_path = os.path.join(output_dir, annotation_file.replace('.json', '.txt'))
    convert_labelme_to_yolo(json_path, output_path)

FileNotFoundError: [Errno 2] No such file or directory: '/content/gdrive/My Drive/dataset/labels'

Passo 4: Treinamento com YOLO

In [5]:
# Criar arquivo train.txt
import glob

image_paths = glob.glob('/content/gdrive/My Drive/dataset/images/*.jpg')
with open('/content/gdrive/My Drive/darknet/train.txt', 'w') as f:
    for path in image_paths:
        f.write(path + '\n')

FileNotFoundError: [Errno 2] No such file or directory: '/content/gdrive/My Drive/darknet/train.txt'

In [6]:
!./darknet detector train cfg/coco.data cfg/yolov3.cfg yolov3.weights

 GPU isn't used 
 OpenCV isn't used - data augmentation will be slow 
yolov3
mini_batch = 1, batch = 1, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5,  wt = 0, wn = 0, outputs: 104 x 104 x 128 0.001 BF
   9 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
  10 conv  

Passo 5:Verificação dos Resultados

In [7]:
!./darknet detector test cfg/coco.data cfg/yolov3.cfg yolov3.weights /content/gdrive/My\ Drive/dataset/images/sample.jpg

 GPU isn't used 
 OpenCV isn't used - data augmentation will be slow 
mini_batch = 1, batch = 1, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5,  wt = 0, wn = 0, outputs: 104 x 104 x 128 0.001 BF
   9 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
  10 conv    128  